## Converting to tflite model

In [3]:
import tensorflow as tf
import numpy as np

# Load the trained Keras model
model = tf.keras.models.load_model('lung_cancer_detection_model.h5')

# Configure the TFLite converter and apply optimizations
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define the representative dataset generator for dual-input
def representative_data_gen():
    for _ in range(100):  # Adjust number of samples as needed
        # Generate random data for each input
        input1 = np.random.rand(1, 224, 224, 3).astype(np.float32)  # ResNet input shape
        input2 = np.random.rand(1, 224, 224, 3).astype(np.float32)  # MobileNet input shape
        yield [input1, input2]  # Yield as a list for two inputs

converter.representative_dataset = representative_data_gen

# Specify full integer quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert and save the optimized model
tflite_model = converter.convert()
with open("lung_cancer_detection_model_optimized.tflite", "wb") as f:
    f.write(tflite_model)

print("Model conversion successful! Optimized TFLite model is saved.")


INFO:tensorflow:Assets written to: /tmp/tmp65tqyd6z/assets


INFO:tensorflow:Assets written to: /tmp/tmp65tqyd6z/assets
/home/sid/anaconda3/envs/tf/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-10-30 16:08:17.703908: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-10-30 16:08:17.703954: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-10-30 16:08:17.704198: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp65tqyd6z
2024-10-30 16:08:17.731693: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-10-30 16:08:17.731730: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp65tqyd6z
2024-10-30 16:08:17.850461: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-10-30 16:

Model conversion successful! Optimized TFLite model is saved.


## Testing

In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="lung_cancer_detection_model_optimized.tflite")
interpreter.allocate_tensors()

# Retrieve input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define class labels
class_labels = {0: "benign", 1: "malignant", 2: "normal"}  # Adjust based on your classes

# Function to preprocess an image for inference
def preprocess_image(image_path):
    image = Image.open(image_path).resize((224, 224))
    input_data = np.array(image, dtype=np.uint8)  # Convert to UINT8 as expected by model
    input_data = np.expand_dims(input_data, axis=0)
    return input_data

# Load and preprocess a sample image
sample_image = preprocess_image("data/train/Malignant cases/Malignant case (1)_aug_0.jpg")

# Set the input tensors (matching dual-input model structure)
interpreter.set_tensor(input_details[0]['index'], sample_image)
interpreter.set_tensor(input_details[1]['index'], sample_image)  # Duplicate input for dual structure

# Run inference
interpreter.invoke()

# Get the output and map to class label
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_index = np.argmax(output_data)
predicted_label = class_labels[predicted_index]
print(f"Predicted Label: {predicted_label}")


2024-10-30 21:52:25.321958: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 21:52:25.356111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 21:52:25.356146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 21:52:25.356935: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 21:52:25.362802: I tensorflow/core/platform/cpu_feature_guar

Predicted Label: malignant


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
